In [1]:
import torch; print(torch.__version__)

0.4.1.post2


In [2]:
import os; import sys
cwd = os.getcwd()
path = os.path.join(cwd, '..', 'movie_recommender')
if not path in sys.path:
    sys.path.append(path)
del cwd, path

In [13]:
from autoencoder import AutoEncoder

In [5]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import pandas as pd
import pickle

import multiprocessing

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [6]:
path = "../data/"
with open('../data/tfidf_matrix.pkl', 'rb') as fh:
    tfidf = pickle.load(fh)

In [7]:
type(tfidf), tfidf.shape

(pandas.core.frame.DataFrame, (26744, 9697))

In [8]:
tfidf.head()

,0,1,2,3,4,5,6,7,8,9,...,9687,9688,9689,9690,9691,9692,9693,9694,9695,9696
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
ae = AutoEncoder(tfidf, validation_perc=0.1, lr=1e-3, intermediate_size=5000, encoded_size=100)

In [17]:
ae.train_loop(epochs=2)

Epoch 1/2
train loss: 0.22211383283138275 | validation loss: 0.22272825241088867)
train loss: 0.20419341325759888 | validation loss: 0.20414476096630096)
train loss: 0.17747719585895538 | validation loss: 0.1798742711544037)
Epoch 2/2
train loss: 0.1487182229757309 | validation loss: 0.15072493255138397)
train loss: 0.13786664605140686 | validation loss: 0.1345129907131195)
train loss: 0.12331461906433105 | validation loss: 0.12234042584896088)


In [18]:
ae.train_loop(epochs=2)

Epoch 1/2
train loss: 0.10038083046674728 | validation loss: 0.10278349369764328)
train loss: 0.09294276684522629 | validation loss: 0.09235644340515137)
train loss: 0.09155938029289246 | validation loss: 0.09073655307292938)
Epoch 2/2
train loss: 0.0769510567188263 | validation loss: 0.07269316166639328)
train loss: 0.0654601901769638 | validation loss: 0.07125481963157654)
train loss: 0.06600766628980637 | validation loss: 0.06300365179777145)


In [19]:
ae.train_loop(epochs=2)

Epoch 1/2
train loss: 0.05271441489458084 | validation loss: 0.05825185030698776)
train loss: 0.05539986118674278 | validation loss: 0.05281775817275047)
train loss: 0.05276575684547424 | validation loss: 0.05088816210627556)
Epoch 2/2
train loss: 0.04165542870759964 | validation loss: 0.04463959485292435)
train loss: 0.045304302126169205 | validation loss: 0.042063675820827484)
train loss: 0.0432794950902462 | validation loss: 0.04143285006284714)


In [20]:
ae.train_loop(epochs=2)

Epoch 1/2
train loss: 0.032550495117902756 | validation loss: 0.0356208011507988)
train loss: 0.0392172671854496 | validation loss: 0.037845008075237274)
train loss: 0.029903411865234375 | validation loss: 0.03419167175889015)
Epoch 2/2
train loss: 0.034451961517333984 | validation loss: 0.03241730108857155)
train loss: 0.026692219078540802 | validation loss: 0.018445396795868874)
train loss: 0.0373699925839901 | validation loss: 0.03099893033504486)


In [21]:
ae.train_loop(epochs=2)

Epoch 1/2
train loss: 0.022262902930378914 | validation loss: 0.02541205659508705)
train loss: 0.020922208204865456 | validation loss: 0.025425033643841743)
train loss: 0.02638252265751362 | validation loss: 0.024518130347132683)
Epoch 2/2
train loss: 0.019866345450282097 | validation loss: 0.020249202847480774)
train loss: 0.015816401690244675 | validation loss: 0.017304060980677605)
train loss: 0.016083482652902603 | validation loss: 0.015688365325331688)


In [22]:
ae.train_loop(epochs=2)

Epoch 1/2
train loss: 0.011935699731111526 | validation loss: 0.012489059939980507)
train loss: 0.011296205222606659 | validation loss: 0.012186301872134209)
train loss: 0.009784495458006859 | validation loss: 0.010611973702907562)
Epoch 2/2
train loss: 0.009321693331003189 | validation loss: 0.008954348042607307)
train loss: 0.007862568832933903 | validation loss: 0.00843227282166481)
train loss: 0.007300532888621092 | validation loss: 0.0076403748244047165)


In [23]:
ae.train_loop(epochs=2)

Epoch 1/2
train loss: 0.00634989095851779 | validation loss: 0.006731168366968632)
train loss: 0.005916875787079334 | validation loss: 0.006326233968138695)
train loss: 0.007038164883852005 | validation loss: 0.0058355312794446945)
Epoch 2/2
train loss: 0.005349320359528065 | validation loss: 0.005119940731674433)
train loss: 0.00465071527287364 | validation loss: 0.0048826164565980434)
train loss: 0.004610786680132151 | validation loss: 0.004540398716926575)


In [24]:
ae.train_loop(epochs=4)

Epoch 1/4
train loss: 0.003916759043931961 | validation loss: 0.004037921316921711)
train loss: 0.0036902856081724167 | validation loss: 0.003838035510852933)
train loss: 0.0035084523260593414 | validation loss: 0.003616831498220563)
Epoch 2/4
train loss: 0.0031521415803581476 | validation loss: 0.0031990164425224066)
train loss: 0.002976352581754327 | validation loss: 0.0030586274806410074)
train loss: 0.0032228981144726276 | validation loss: 0.0029321531765162945)
Epoch 3/4
train loss: 0.00267465366050601 | validation loss: 0.0026465607807040215)
train loss: 0.0024370907340198755 | validation loss: 0.0024968425277620554)
train loss: 0.0024052713997662067 | validation loss: 0.002401751698926091)
Epoch 4/4
train loss: 0.002114248462021351 | validation loss: 0.0021863819565624)
train loss: 0.0020306806545704603 | validation loss: 0.0020791387651115656)
train loss: 0.0019213099731132388 | validation loss: 0.0019558239728212357)


In [25]:
ae.train_loop(epochs=4)

Epoch 1/4
train loss: 0.0018042513402178884 | validation loss: 0.0017797506880015135)
train loss: 0.0016782403690740466 | validation loss: 0.0017056285869330168)
train loss: 0.0015953808324411511 | validation loss: 0.001635564025491476)
Epoch 2/4
train loss: 0.0014724330976605415 | validation loss: 0.0014963321154937148)
train loss: 0.0014251103857532144 | validation loss: 0.0014283841010183096)
train loss: 0.0013734757667407393 | validation loss: 0.001365144271403551)
Epoch 3/4
train loss: 0.0012548015220090747 | validation loss: 0.0012641212670132518)
train loss: 0.0012136232107877731 | validation loss: 0.001206468790769577)
train loss: 0.0011289498070254922 | validation loss: 0.0011448214063420892)
Epoch 4/4
train loss: 0.001059606671333313 | validation loss: 0.0010619742097333074)
train loss: 0.0010363450273871422 | validation loss: 0.0010200743563473225)
train loss: 0.000986527418717742 | validation loss: 0.0009731805184856057)


In [26]:
ae.train_loop(epochs=4)

Epoch 1/4
train loss: 0.0008942043641582131 | validation loss: 0.0008976528770290315)
train loss: 0.0008707244996912777 | validation loss: 0.0008679538732394576)
train loss: 0.000838449748698622 | validation loss: 0.0008310682023875415)
Epoch 2/4
train loss: 0.0007735221297480166 | validation loss: 0.0007736966945230961)
train loss: 0.0007494702003896236 | validation loss: 0.0007416484877467155)
train loss: 0.0007115761982277036 | validation loss: 0.00071835977723822)
Epoch 3/4
train loss: 0.0006806914461776614 | validation loss: 0.0006709630251862109)
train loss: 0.0006495128618553281 | validation loss: 0.00064698705682531)
train loss: 0.0006236248300410807 | validation loss: 0.0006236637127585709)
Epoch 4/4
train loss: 0.000588148832321167 | validation loss: 0.0005896996008232236)
train loss: 0.0005583107704296708 | validation loss: 0.0005680866306647658)
train loss: 0.0005578541895374656 | validation loss: 0.0005491554038599133)


In [27]:
ae.train_loop(epochs=4)

Epoch 1/4
train loss: 0.0005287472740747035 | validation loss: 0.0005193562246859074)
train loss: 0.0005033314228057861 | validation loss: 0.0005030590109527111)
train loss: 0.00048805269761942327 | validation loss: 0.00048723051440902054)
Epoch 2/4
train loss: 0.0004674179363064468 | validation loss: 0.0004637388919945806)
train loss: 0.0004540400404948741 | validation loss: 0.0004503401869442314)
train loss: 0.00044429569970816374 | validation loss: 0.0004385995853226632)
Epoch 3/4
train loss: 0.00041791549301706254 | validation loss: 0.00041860347846522927)
train loss: 0.0004058197664562613 | validation loss: 0.00040799405542202294)
train loss: 0.00040128841646946967 | validation loss: 0.0003976552397944033)
Epoch 4/4
train loss: 0.00038252287777140737 | validation loss: 0.0003824015730060637)
train loss: 0.00038215192034840584 | validation loss: 0.0003739483072422445)
train loss: 0.00036855138023383915 | validation loss: 0.00036536940024234354)


In [32]:
encoded = ae.get_encoded_representations()

In [33]:
encoded.shape

(26744, 100)

In [34]:
encoded[:10, :5]

array([[0.05032459, 0.        , 0.        , 0.        , 0.53481305],
       [0.        , 0.        , 0.        , 0.        , 0.46456537],
       [0.        , 0.        , 0.        , 0.        , 0.5692221 ],
       [0.00185815, 0.        , 0.        , 0.        , 0.43467408],
       [0.        , 0.        , 0.        , 0.        , 0.4278627 ],
       [0.2352247 , 0.        , 0.        , 0.        , 0.47872636],
       [0.10936389, 0.        , 0.        , 0.        , 0.52176124],
       [0.04019884, 0.        , 0.        , 0.        , 0.47893742],
       [0.2287716 , 0.        , 0.        , 0.        , 0.44490448],
       [0.        , 0.        , 0.        , 0.        , 0.54743814]],
      dtype=float32)

In [35]:
with open('../data/autoencoder_embeddings.pkl', 'wb') as fh:
    pickle.dump(encoded, fh)